In [1]:
%pylab inline
import pandas as pd
import numpy as np
from pandas import Series, DataFrame
import re
import datetime as dt
import matplotlib.pyplot as plt
import time
import seaborn as sns

Populating the interactive namespace from numpy and matplotlib


In [2]:
def ht(df, n = 5):
    print(df.shape)
    if len(df) <= 2*n:
        display(df)
    else:
        display(df.head(n).append(df.tail(n)))
def pf(df):
    pd.set_option('display.max_rows', len(df))
    display(df)
    pd.reset_option('display.max_rows')

In [3]:
pd.set_option('display.max_columns', 100)
plt.rc('figure', figsize=(13, 7))
plt.style.use('ggplot')

In [4]:
from sklearn import datasets

In [5]:
from sklearn import preprocessing
from sklearn import metrics
from sklearn.model_selection import train_test_split

In [13]:
from sklearn.preprocessing import StandardScaler

In [6]:
breast_cancer = datasets.load_breast_cancer()

In [7]:
breast_cancer.data.shape

(569, 30)

In [9]:
digits = datasets.load_digits()

In [10]:
print(digits.DESCR)

.. _digits_dataset:

Optical recognition of handwritten digits dataset
--------------------------------------------------

**Data Set Characteristics:**

    :Number of Instances: 5620
    :Number of Attributes: 64
    :Attribute Information: 8x8 image of integer pixels in the range 0..16.
    :Missing Attribute Values: None
    :Creator: E. Alpaydin (alpaydin '@' boun.edu.tr)
    :Date: July; 1998

This is a copy of the test set of the UCI ML hand-written digits datasets
http://archive.ics.uci.edu/ml/datasets/Optical+Recognition+of+Handwritten+Digits

The data set contains images of hand-written digits: 10 classes where
each class refers to a digit.

Preprocessing programs made available by NIST were used to extract
normalized bitmaps of handwritten digits from a preprinted form. From a
total of 43 people, 30 contributed to the training set and different 13
to the test set. 32x32 bitmaps are divided into nonoverlapping blocks of
4x4 and the number of on pixels are counted in each bloc

In [11]:
digits.data.shape

(1797, 64)

In [16]:
seed = 7

In [26]:
from collections import Counter

In [27]:
def is_balanced(seq):
    n = len(seq)
    classes = [(clas, float(count)) for clas, count in Counter(seq).items()]
    k = len(classes)

    H = -sum([(count/n) * np.log((count/n)) for clas, count in classes])
    return H/np.log(k) > 0.75

In [28]:
is_balanced(digits.target)

True

In [29]:
is_balanced(breast_cancer.target)

True

In [63]:
import sklearn.model_selection as ms

In [64]:
features = digits.data
classes = digits.target

In [65]:
test_size = 0.3

In [66]:
training_x, testing_x, training_y, testing_y = ms.train_test_split(
    features, classes, test_size=test_size, random_state=seed, stratify=classes
)

In [68]:
print(training_x.shape)
print(testing_x.shape)

(1257, 64)
(540, 64)


In [69]:
scaler = StandardScaler().fit(training_x)

In [70]:
training_x

array([[ 0.,  0.,  4., ...,  5.,  0.,  0.],
       [ 0.,  0.,  3., ..., 16., 16., 16.],
       [ 0.,  0.,  4., ..., 11.,  0.,  0.],
       ...,
       [ 0.,  0., 13., ..., 11.,  1.,  0.],
       [ 0.,  1., 10., ...,  7.,  0.,  0.],
       [ 0.,  0.,  3., ...,  8.,  0.,  0.]])

In [71]:
training_x = scaler.transform(training_x)

In [72]:
testing_x = scaler.transform(testing_x)

## Baseline

In [75]:
from sklearn.neural_network import MLPClassifier

In [76]:
from sklearn.pipeline import Pipeline

In [77]:
_nn_arch = [(50, 50), (50,), (25,), (25, 25), (100, 25, 100)]
_nn_reg = [10 ** -x for x in range(1, 5)]

In [78]:
grid = {'NN__alpha': _nn_reg, 'NN__hidden_layer_sizes': _nn_arch}
mlp = MLPClassifier(activation='relu', max_iter=2000, early_stopping=True, random_state=seed)
pipe = Pipeline([('NN', mlp)])

In [79]:
from sklearn.model_selection import GridSearchCV

In [80]:
from sklearn.metrics import make_scorer, accuracy_score, f1_score

In [81]:
from sklearn.utils import compute_sample_weight

In [82]:
def balanced_accuracy(truth, pred):
    wts = compute_sample_weight('balanced', truth)
    return accuracy_score(truth, pred, sample_weight=wts)

scorer = make_scorer(balanced_accuracy)

In [83]:
def gs_with_best_estimator(pipe, grid):
    gs = GridSearchCV(pipe, grid, verbose=10, cv=5, scoring=scorer)
    gs.fit(training_x, training_y)

    best_estimator = gs.best_estimator_.fit(training_x, training_y)
    final_estimator = best_estimator._final_estimator
    best_params = pd.DataFrame([best_estimator.get_params()])
    final_estimator_params = pd.DataFrame([final_estimator.get_params()])

    return gs, best_estimator, final_estimator

In [84]:
gs, best_estimator, final_estimator = gs_with_best_estimator(pipe, grid)

Fitting 5 folds for each of 20 candidates, totalling 100 fits
[CV] NN__alpha=0.1, NN__hidden_layer_sizes=(50, 50) ..................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  NN__alpha=0.1, NN__hidden_layer_sizes=(50, 50), score=0.9455384615384614, total=   0.4s
[CV] NN__alpha=0.1, NN__hidden_layer_sizes=(50, 50) ..................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.4s remaining:    0.0s


[CV]  NN__alpha=0.1, NN__hidden_layer_sizes=(50, 50), score=0.9564615384615385, total=   0.3s
[CV] NN__alpha=0.1, NN__hidden_layer_sizes=(50, 50) ..................


...

[CV]  NN__alpha=0.1, NN__hidden_layer_sizes=(50,), score=0.9191666666666668, total=   0.2s
[CV] NN__alpha=0.1, NN__hidden_layer_sizes=(50,) .....................
[CV]  NN__alpha=0.1, NN__hidden_layer_sizes=(50,), score=0.947, total=   0.2s
[CV] NN__alpha=0.1, NN__hidden_layer_sizes=(25,) .....................


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:    2.2s remaining:    0.0s


[CV]  NN__alpha=0.1, NN__hidden_layer_sizes=(25,), score=0.8944615384615384, total=   0.2s
[CV] NN__alpha=0.1, NN__hidden_layer_sizes=(25,) .....................
[CV]  NN__alpha=0.1, NN__hidden_layer_sizes=(25,), score=0.8572307692307694, total=   0.2s
[CV] NN__alpha=0.1, NN__hidden_layer_sizes=(25,) .....................
[CV]  NN__alpha=0.1, NN__hidden_layer_sizes=(25,), score=0.9638333333333332, total=   0.2s
[CV] NN__alpha=0.1, NN__hidden_layer_sizes=(25,) .....................
[CV]  NN__alpha=0.1, NN__hidden_layer_sizes=(25,), score=0.9110000000000001, total=   0.2s
[CV] NN__alpha=0.1, NN__hidden_layer_sizes=(25,) .....................
[CV]  NN__alpha=0.1, NN__hidden_layer_sizes=(25,), score=0.9595000000000001, total=   0.2s
[CV] NN__alpha=0.1, NN__hidden_layer_sizes=(25, 25) ..................
[CV]  NN__alpha=0.1, NN__hidden_layer_sizes=(25, 25), score=0.9332307692307691, total=   0.3s
[CV] NN__alpha=0.1, NN__hidden_layer_sizes=(25, 25) ..................
[CV]  NN__alpha=0.1, NN__

[CV]  NN__alpha=0.001, NN__hidden_layer_sizes=(25,), score=0.8944615384615384, total=   0.2s
[CV] NN__alpha=0.001, NN__hidden_layer_sizes=(25,) ...................
[CV]  NN__alpha=0.001, NN__hidden_layer_sizes=(25,), score=0.8572307692307694, total=   0.1s
[CV] NN__alpha=0.001, NN__hidden_layer_sizes=(25,) ...................
[CV]  NN__alpha=0.001, NN__hidden_layer_sizes=(25,), score=0.9638333333333332, total=   0.2s
[CV] NN__alpha=0.001, NN__hidden_layer_sizes=(25,) ...................
[CV]  NN__alpha=0.001, NN__hidden_layer_sizes=(25,), score=0.915, total=   0.2s
[CV] NN__alpha=0.001, NN__hidden_layer_sizes=(25,) ...................
[CV]  NN__alpha=0.001, NN__hidden_layer_sizes=(25,), score=0.9595000000000001, total=   0.2s
[CV] NN__alpha=0.001, NN__hidden_layer_sizes=(25, 25) ................
[CV]  NN__alpha=0.001, NN__hidden_layer_sizes=(25, 25), score=0.9332307692307691, total=   0.3s
[CV] NN__alpha=0.001, NN__hidden_layer_sizes=(25, 25) ................
[CV]  NN__alpha=0.001, NN_

[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:   27.2s finished
/Users/frankz/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [87]:
best_estimator

Pipeline(memory=None,
     steps=[('NN', MLPClassifier(activation='relu', alpha=0.001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=True, epsilon=1e-08,
       hidden_layer_sizes=(100, 25, 100), learning_rate='constant',
       learning_rate_init=0.001, max_iter=2000, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=7, shuffle=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False))])

In [89]:
best_estimator.score(training_x, training_y)

0.9936356404136834

In [86]:
best_estimator.score(testing_x, testing_y)

0.9611111111111111

## Dimensionality Reduction

### PCA

In [100]:
from sklearn.decomposition import PCA

In [101]:
pca = PCA(n_components=8, random_state=seed)

In [103]:
grid = {'NN__alpha': _nn_reg, 'NN__hidden_layer_sizes': _nn_arch}
mlp = MLPClassifier(activation='relu', max_iter=2000, early_stopping=True, random_state=seed)
pipe = Pipeline([('pca', pca), ('NN', mlp)])

In [105]:
gs, best_estimator, final_estimator = gs_with_best_estimator(pipe, grid)

Fitting 5 folds for each of 20 candidates, totalling 100 fits
[CV] NN__alpha=0.1, NN__hidden_layer_sizes=(50, 50) ..................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  NN__alpha=0.1, NN__hidden_layer_sizes=(50, 50), score=0.8470769230769231, total=   0.2s
[CV] NN__alpha=0.1, NN__hidden_layer_sizes=(50, 50) ..................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.2s remaining:    0.0s


[CV]  NN__alpha=0.1, NN__hidden_layer_sizes=(50, 50), score=0.8772307692307694, total=   0.5s
[CV] NN__alpha=0.1, NN__hidden_layer_sizes=(50, 50) ..................


...

[CV]  NN__alpha=0.1, NN__hidden_layer_sizes=(50,), score=0.8460000000000001, total=   0.2s
[CV] NN__alpha=0.1, NN__hidden_layer_sizes=(50,) .....................
[CV]  NN__alpha=0.1, NN__hidden_layer_sizes=(50,), score=0.8365, total=   0.2s
[CV] NN__alpha=0.1, NN__hidden_layer_sizes=(25,) .....................


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:    2.3s remaining:    0.0s


[CV]  NN__alpha=0.1, NN__hidden_layer_sizes=(25,), score=0.8036923076923077, total=   0.2s
[CV] NN__alpha=0.1, NN__hidden_layer_sizes=(25,) .....................
[CV]  NN__alpha=0.1, NN__hidden_layer_sizes=(25,), score=0.8255384615384617, total=   0.2s
[CV] NN__alpha=0.1, NN__hidden_layer_sizes=(25,) .....................
[CV]  NN__alpha=0.1, NN__hidden_layer_sizes=(25,), score=0.7852435897435898, total=   0.2s
[CV] NN__alpha=0.1, NN__hidden_layer_sizes=(25,) .....................
[CV]  NN__alpha=0.1, NN__hidden_layer_sizes=(25,), score=0.7280000000000001, total=   0.1s
[CV] NN__alpha=0.1, NN__hidden_layer_sizes=(25,) .....................
[CV]  NN__alpha=0.1, NN__hidden_layer_sizes=(25,), score=0.8248333333333334, total=   0.2s
[CV] NN__alpha=0.1, NN__hidden_layer_sizes=(25, 25) ..................
[CV]  NN__alpha=0.1, NN__hidden_layer_sizes=(25, 25), score=0.8036923076923076, total=   0.2s
[CV] NN__alpha=0.1, NN__hidden_layer_sizes=(25, 25) ..................
[CV]  NN__alpha=0.1, NN__

[CV]  NN__alpha=0.001, NN__hidden_layer_sizes=(25,), score=0.8036923076923077, total=   0.2s
[CV] NN__alpha=0.001, NN__hidden_layer_sizes=(25,) ...................
[CV]  NN__alpha=0.001, NN__hidden_layer_sizes=(25,), score=0.8255384615384617, total=   0.3s
[CV] NN__alpha=0.001, NN__hidden_layer_sizes=(25,) ...................
[CV]  NN__alpha=0.001, NN__hidden_layer_sizes=(25,), score=0.7852435897435898, total=   0.2s
[CV] NN__alpha=0.001, NN__hidden_layer_sizes=(25,) ...................
[CV]  NN__alpha=0.001, NN__hidden_layer_sizes=(25,), score=0.7280000000000001, total=   0.1s
[CV] NN__alpha=0.001, NN__hidden_layer_sizes=(25,) ...................
[CV]  NN__alpha=0.001, NN__hidden_layer_sizes=(25,), score=0.8248333333333334, total=   0.2s
[CV] NN__alpha=0.001, NN__hidden_layer_sizes=(25, 25) ................
[CV]  NN__alpha=0.001, NN__hidden_layer_sizes=(25, 25), score=0.8075384615384615, total=   0.2s
[CV] NN__alpha=0.001, NN__hidden_layer_sizes=(25, 25) ................
[CV]  NN__alp

[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:   24.4s finished


In [110]:
final_estimator

MLPClassifier(activation='relu', alpha=0.001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=True, epsilon=1e-08,
       hidden_layer_sizes=(100, 25, 100), learning_rate='constant',
       learning_rate_init=0.001, max_iter=2000, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=7, shuffle=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False)

In [107]:
best_estimator.score(training_x, training_y)

0.9172633253778838

In [108]:
best_estimator.score(testing_x, testing_y)

0.8851851851851852

### ICA

In [113]:
from sklearn.decomposition import FastICA

In [114]:
ica = FastICA(n_components=13, random_state=seed)

In [115]:
grid = {'NN__alpha': _nn_reg, 'NN__hidden_layer_sizes': _nn_arch}
mlp = MLPClassifier(activation='relu', max_iter=2000, early_stopping=True, random_state=seed)
pipe = Pipeline([('ica', ica), ('NN', mlp)])

In [116]:
gs, best_estimator, final_estimator = gs_with_best_estimator(pipe, grid)

Fitting 5 folds for each of 20 candidates, totalling 100 fits
[CV] NN__alpha=0.1, NN__hidden_layer_sizes=(50, 50) ..................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/Users/frankz/anaconda3/lib/python3.7/site-packages/sklearn/decomposition/fastica_.py:121: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  ConvergenceWarning)
/Users/frankz/anaconda3/lib/python3.7/site-packages/scipy/linalg/basic.py:1321: RuntimeWarning: internal gelsd driver lwork query error, required iwork dimension not returned. This is likely the result of LAPACK bug 0038, fixed in LAPACK 3.2.2 (released July 21, 2010). Falling back to 'gelss' driver.
  x, resids, rank, s = lstsq(a, b, cond=cond, check_finite=False)


[CV]  NN__alpha=0.1, NN__hidden_layer_sizes=(50, 50), score=0.8192307692307692, total=   0.6s
[CV] NN__alpha=0.1, NN__hidden_layer_sizes=(50, 50) ..................
[CV]  NN__alpha=0.1, NN__hidden_layer_sizes=(50, 50), score=0.18461538461538465, total=   0.2s
[CV] NN__alpha=0.1, NN__hidden_layer_sizes=(50, 50) ..................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.6s remaining:    0.0s
/Users/frankz/anaconda3/lib/python3.7/site-packages/sklearn/decomposition/fastica_.py:121: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  ConvergenceWarning)
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.8s remaining:    0.0s
/Users/frankz/anaconda3/lib/python3.7/site-packages/sklearn/decomposition/fastica_.py:121: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  ConvergenceWarning)


[CV]  NN__alpha=0.1, NN__hidden_layer_sizes=(50, 50), score=0.6690897435897436, total=   0.4s
[CV] NN__alpha=0.1, NN__hidden_layer_sizes=(50, 50) ..................
[CV]  NN__alpha=0.1, NN__hidden_layer_sizes=(50, 50), score=0.09999999999999999, total=   0.2s
[CV] NN__alpha=0.1, NN__hidden_layer_sizes=(50, 50) ..................


...

/Users/frankz/anaconda3/lib/python3.7/site-packages/sklearn/decomposition/fastica_.py:121: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  ConvergenceWarning)


[CV]  NN__alpha=0.0001, NN__hidden_layer_sizes=(100, 25, 100), score=0.7856410256410257, total=   0.9s
[CV] NN__alpha=0.0001, NN__hidden_layer_sizes=(100, 25, 100) .........


/Users/frankz/anaconda3/lib/python3.7/site-packages/sklearn/decomposition/fastica_.py:121: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  ConvergenceWarning)


[CV]  NN__alpha=0.0001, NN__hidden_layer_sizes=(100, 25, 100), score=0.6920000000000002, total=   0.5s
[CV] NN__alpha=0.0001, NN__hidden_layer_sizes=(100, 25, 100) .........
[CV]  NN__alpha=0.0001, NN__hidden_layer_sizes=(100, 25, 100), score=0.5355000000000001, total=   0.3s


[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:   27.8s finished
/Users/frankz/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
/Users/frankz/anaconda3/lib/python3.7/site-packages/sklearn/decomposition/fastica_.py:121: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  ConvergenceWarning)
/Users/frankz/anaconda3/lib/python3.7/site-packages/sklearn/decomposition/fastica_.py:121: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  ConvergenceWarning)


In [117]:
final_estimator

MLPClassifier(activation='relu', alpha=0.01, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=True, epsilon=1e-08,
       hidden_layer_sizes=(50, 50), learning_rate='constant',
       learning_rate_init=0.001, max_iter=2000, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=7, shuffle=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False)

In [118]:
best_estimator.score(training_x, training_y)

0.8870326173428799

In [119]:
best_estimator.score(testing_x, testing_y)

0.8944444444444445

### RP

In [123]:
from sklearn.random_projection import SparseRandomProjection, GaussianRandomProjection

In [124]:
rp = SparseRandomProjection(n_components=9, random_state=seed)

In [125]:
grid = {'NN__alpha': _nn_reg, 'NN__hidden_layer_sizes': _nn_arch}
mlp = MLPClassifier(activation='relu', max_iter=2000, early_stopping=True, random_state=seed)
pipe = Pipeline([('rp', rp), ('NN', mlp)])

In [126]:
gs, best_estimator, final_estimator = gs_with_best_estimator(pipe, grid)

Fitting 5 folds for each of 20 candidates, totalling 100 fits
[CV] NN__alpha=0.1, NN__hidden_layer_sizes=(50, 50) ..................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  NN__alpha=0.1, NN__hidden_layer_sizes=(50, 50), score=0.7459999999999999, total=   0.3s
[CV] NN__alpha=0.1, NN__hidden_layer_sizes=(50, 50) ..................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.3s remaining:    0.0s


[CV]  NN__alpha=0.1, NN__hidden_layer_sizes=(50, 50), score=0.7713846153846156, total=   0.3s
[CV] NN__alpha=0.1, NN__hidden_layer_sizes=(50, 50) ..................


...

[CV]  NN__alpha=0.1, NN__hidden_layer_sizes=(50,), score=0.762, total=   0.3s
[CV] NN__alpha=0.1, NN__hidden_layer_sizes=(50,) .....................


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:    2.6s remaining:    0.0s


[CV]  NN__alpha=0.1, NN__hidden_layer_sizes=(50,), score=0.7166666666666668, total=   0.2s
[CV] NN__alpha=0.1, NN__hidden_layer_sizes=(25,) .....................
[CV]  NN__alpha=0.1, NN__hidden_layer_sizes=(25,), score=0.6561538461538461, total=   0.2s
[CV] NN__alpha=0.1, NN__hidden_layer_sizes=(25,) .....................
[CV]  NN__alpha=0.1, NN__hidden_layer_sizes=(25,), score=0.6432307692307693, total=   0.2s
[CV] NN__alpha=0.1, NN__hidden_layer_sizes=(25,) .....................
[CV]  NN__alpha=0.1, NN__hidden_layer_sizes=(25,), score=0.699076923076923, total=   0.2s
[CV] NN__alpha=0.1, NN__hidden_layer_sizes=(25,) .....................
[CV]  NN__alpha=0.1, NN__hidden_layer_sizes=(25,), score=0.6258333333333335, total=   0.2s
[CV] NN__alpha=0.1, NN__hidden_layer_sizes=(25,) .....................
[CV]  NN__alpha=0.1, NN__hidden_layer_sizes=(25,), score=0.7130000000000001, total=   0.2s
[CV] NN__alpha=0.1, NN__hidden_layer_sizes=(25, 25) ..................
[CV]  NN__alpha=0.1, NN__hidd

[CV]  NN__alpha=0.001, NN__hidden_layer_sizes=(50,), score=0.7208333333333333, total=   0.2s
[CV] NN__alpha=0.001, NN__hidden_layer_sizes=(25,) ...................
[CV]  NN__alpha=0.001, NN__hidden_layer_sizes=(25,), score=0.6561538461538461, total=   0.2s
[CV] NN__alpha=0.001, NN__hidden_layer_sizes=(25,) ...................
[CV]  NN__alpha=0.001, NN__hidden_layer_sizes=(25,), score=0.6432307692307693, total=   0.2s
[CV] NN__alpha=0.001, NN__hidden_layer_sizes=(25,) ...................
[CV]  NN__alpha=0.001, NN__hidden_layer_sizes=(25,), score=0.699076923076923, total=   0.1s
[CV] NN__alpha=0.001, NN__hidden_layer_sizes=(25,) ...................
[CV]  NN__alpha=0.001, NN__hidden_layer_sizes=(25,), score=0.6258333333333335, total=   0.1s
[CV] NN__alpha=0.001, NN__hidden_layer_sizes=(25,) ...................
[CV]  NN__alpha=0.001, NN__hidden_layer_sizes=(25,), score=0.7331666666666666, total=   0.2s
[CV] NN__alpha=0.001, NN__hidden_layer_sizes=(25, 25) ................
[CV]  NN__alpha=0

[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:   30.7s finished
/Users/frankz/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [127]:
final_estimator

MLPClassifier(activation='relu', alpha=0.001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=True, epsilon=1e-08,
       hidden_layer_sizes=(100, 25, 100), learning_rate='constant',
       learning_rate_init=0.001, max_iter=2000, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=7, shuffle=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False)

In [128]:
best_estimator.score(training_x, training_y)

0.883054892601432

In [129]:
best_estimator.score(testing_x, testing_y)

0.8185185185185185

### RF

In [134]:
from sklearn.ensemble import RandomForestClassifier

In [135]:
from sklearn.base import TransformerMixin,BaseEstimator

In [136]:
class ImportanceSelect(BaseEstimator, TransformerMixin):
    def __init__(self, model, n=1):
        self.model = model
        self.n = n

    def fit(self, *args, **kwargs):
        self.model.fit(*args, **kwargs)
        return self

    def transform(self, X):
        return X[:, self.model.feature_importances_.argsort()[::-1][:self.n]]

In [137]:
rfc = RandomForestClassifier(n_estimators=100, class_weight='balanced', random_state=seed)
filtr = ImportanceSelect(rfc, 11)

In [138]:
grid = {'NN__alpha': _nn_reg, 'NN__hidden_layer_sizes': _nn_arch}
mlp = MLPClassifier(activation='relu', max_iter=2000, early_stopping=True, random_state=seed)
pipe = Pipeline([('filter', filtr), ('NN', mlp)])

In [139]:
gs, best_estimator, final_estimator = gs_with_best_estimator(pipe, grid)

Fitting 5 folds for each of 20 candidates, totalling 100 fits
[CV] NN__alpha=0.1, NN__hidden_layer_sizes=(50, 50) ..................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  NN__alpha=0.1, NN__hidden_layer_sizes=(50, 50), score=0.8513846153846154, total=   0.5s
[CV] NN__alpha=0.1, NN__hidden_layer_sizes=(50, 50) ..................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.5s remaining:    0.0s


[CV]  NN__alpha=0.1, NN__hidden_layer_sizes=(50, 50), score=0.8243076923076924, total=   0.4s
[CV] NN__alpha=0.1, NN__hidden_layer_sizes=(50, 50) ..................


...

[CV]  NN__alpha=0.1, NN__hidden_layer_sizes=(50,), score=0.7981666666666667, total=   0.3s
[CV] NN__alpha=0.1, NN__hidden_layer_sizes=(50,) .....................


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:    3.8s remaining:    0.0s


[CV]  NN__alpha=0.1, NN__hidden_layer_sizes=(50,), score=0.7653333333333334, total=   0.3s
[CV] NN__alpha=0.1, NN__hidden_layer_sizes=(25,) .....................
[CV]  NN__alpha=0.1, NN__hidden_layer_sizes=(25,), score=0.796153846153846, total=   0.3s
[CV] NN__alpha=0.1, NN__hidden_layer_sizes=(25,) .....................
[CV]  NN__alpha=0.1, NN__hidden_layer_sizes=(25,), score=0.7646153846153847, total=   0.3s
[CV] NN__alpha=0.1, NN__hidden_layer_sizes=(25,) .....................
[CV]  NN__alpha=0.1, NN__hidden_layer_sizes=(25,), score=0.7948205128205129, total=   0.4s
[CV] NN__alpha=0.1, NN__hidden_layer_sizes=(25,) .....................
[CV]  NN__alpha=0.1, NN__hidden_layer_sizes=(25,), score=0.7655000000000001, total=   0.3s
[CV] NN__alpha=0.1, NN__hidden_layer_sizes=(25,) .....................
[CV]  NN__alpha=0.1, NN__hidden_layer_sizes=(25,), score=0.8301666666666667, total=   0.3s
[CV] NN__alpha=0.1, NN__hidden_layer_sizes=(25, 25) ..................
[CV]  NN__alpha=0.1, NN__hidd

[CV]  NN__alpha=0.001, NN__hidden_layer_sizes=(50,), score=0.7653333333333334, total=   0.3s
[CV] NN__alpha=0.001, NN__hidden_layer_sizes=(25,) ...................
[CV]  NN__alpha=0.001, NN__hidden_layer_sizes=(25,), score=0.796153846153846, total=   0.3s
[CV] NN__alpha=0.001, NN__hidden_layer_sizes=(25,) ...................
[CV]  NN__alpha=0.001, NN__hidden_layer_sizes=(25,), score=0.7646153846153847, total=   0.3s
[CV] NN__alpha=0.001, NN__hidden_layer_sizes=(25,) ...................
[CV]  NN__alpha=0.001, NN__hidden_layer_sizes=(25,), score=0.7948205128205129, total=   0.3s
[CV] NN__alpha=0.001, NN__hidden_layer_sizes=(25,) ...................
[CV]  NN__alpha=0.001, NN__hidden_layer_sizes=(25,), score=0.7655000000000001, total=   0.3s
[CV] NN__alpha=0.001, NN__hidden_layer_sizes=(25,) ...................
[CV]  NN__alpha=0.001, NN__hidden_layer_sizes=(25,), score=0.8301666666666667, total=   0.3s
[CV] NN__alpha=0.001, NN__hidden_layer_sizes=(25, 25) ................
[CV]  NN__alpha=0

[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:   42.5s finished
/Users/frankz/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [140]:
final_estimator

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=True, epsilon=1e-08,
       hidden_layer_sizes=(100, 25, 100), learning_rate='constant',
       learning_rate_init=0.001, max_iter=2000, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=7, shuffle=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False)

In [141]:
best_estimator.score(training_x, training_y)

0.9371519490851233

In [142]:
best_estimator.score(testing_x, testing_y)

0.9055555555555556

## Clustering after DR

### PCA

In [148]:
from sklearn.decomposition import PCA

In [160]:
from sklearn.cluster import KMeans as kmeans
from sklearn.mixture import GaussianMixture as GMM
class CustomGMM(GMM):
    def transform(self, x):
        return self.predict_proba(x)

In [161]:
grid = {'NN__alpha': _nn_reg, 'NN__hidden_layer_sizes': _nn_arch}
mlp = MLPClassifier(activation='relu', max_iter=2000, early_stopping=True, random_state=seed)
pca = PCA(n_components=8, random_state=seed)
km = kmeans(random_state=seed)
km.set_params(n_clusters=10)
gmm = CustomGMM(random_state=seed)
gmm.set_params(n_components=10)

CustomGMM(covariance_type='full', init_params='kmeans', max_iter=100,
     means_init=None, n_components=10, n_init=1, precisions_init=None,
     random_state=7, reg_covar=1e-06, tol=0.001, verbose=0,
     verbose_interval=10, warm_start=False, weights_init=None)

In [151]:
pipe = Pipeline([('pca', pca), ('km', km), ('NN', mlp)])
gs, best_estimator, final_estimator = gs_with_best_estimator(pipe, grid)

Fitting 5 folds for each of 20 candidates, totalling 100 fits
[CV] NN__alpha=0.1, NN__hidden_layer_sizes=(50, 50) ..................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  NN__alpha=0.1, NN__hidden_layer_sizes=(50, 50), score=0.800153846153846, total=   0.5s
[CV] NN__alpha=0.1, NN__hidden_layer_sizes=(50, 50) ..................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.5s remaining:    0.0s


[CV]  NN__alpha=0.1, NN__hidden_layer_sizes=(50, 50), score=0.7790769230769232, total=   0.3s
[CV] NN__alpha=0.1, NN__hidden_layer_sizes=(50, 50) ..................


...

[CV]  NN__alpha=0.1, NN__hidden_layer_sizes=(50,), score=0.8109999999999999, total=   0.4s
[CV] NN__alpha=0.1, NN__hidden_layer_sizes=(50,) .....................


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:    3.2s remaining:    0.0s


[CV]  NN__alpha=0.1, NN__hidden_layer_sizes=(50,), score=0.8585, total=   0.5s
[CV] NN__alpha=0.1, NN__hidden_layer_sizes=(25,) .....................
[CV]  NN__alpha=0.1, NN__hidden_layer_sizes=(25,), score=0.15999999999999998, total=   0.1s
[CV] NN__alpha=0.1, NN__hidden_layer_sizes=(25,) .....................
[CV]  NN__alpha=0.1, NN__hidden_layer_sizes=(25,), score=0.3003076923076924, total=   0.2s
[CV] NN__alpha=0.1, NN__hidden_layer_sizes=(25,) .....................
[CV]  NN__alpha=0.1, NN__hidden_layer_sizes=(25,), score=0.1, total=   0.1s
[CV] NN__alpha=0.1, NN__hidden_layer_sizes=(25,) .....................
[CV]  NN__alpha=0.1, NN__hidden_layer_sizes=(25,), score=0.129, total=   0.1s
[CV] NN__alpha=0.1, NN__hidden_layer_sizes=(25,) .....................
[CV]  NN__alpha=0.1, NN__hidden_layer_sizes=(25,), score=0.08016666666666669, total=   0.2s
[CV] NN__alpha=0.1, NN__hidden_layer_sizes=(25, 25) ..................
[CV]  NN__alpha=0.1, NN__hidden_layer_sizes=(25, 25), score=0.1466

[CV]  NN__alpha=0.001, NN__hidden_layer_sizes=(25,), score=0.3003076923076924, total=   0.2s
[CV] NN__alpha=0.001, NN__hidden_layer_sizes=(25,) ...................
[CV]  NN__alpha=0.001, NN__hidden_layer_sizes=(25,), score=0.1, total=   0.1s
[CV] NN__alpha=0.001, NN__hidden_layer_sizes=(25,) ...................
[CV]  NN__alpha=0.001, NN__hidden_layer_sizes=(25,), score=0.129, total=   0.1s
[CV] NN__alpha=0.001, NN__hidden_layer_sizes=(25,) ...................
[CV]  NN__alpha=0.001, NN__hidden_layer_sizes=(25,), score=0.08016666666666669, total=   0.1s
[CV] NN__alpha=0.001, NN__hidden_layer_sizes=(25, 25) ................
[CV]  NN__alpha=0.001, NN__hidden_layer_sizes=(25, 25), score=0.7964615384615383, total=   0.4s
[CV] NN__alpha=0.001, NN__hidden_layer_sizes=(25, 25) ................
[CV]  NN__alpha=0.001, NN__hidden_layer_sizes=(25, 25), score=0.7512307692307694, total=   0.2s
[CV] NN__alpha=0.001, NN__hidden_layer_sizes=(25, 25) ................
[CV]  NN__alpha=0.001, NN__hidden_lay

[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:   32.9s finished
/Users/frankz/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [152]:
best_estimator

Pipeline(memory=None,
     steps=[('pca', PCA(copy=True, iterated_power='auto', n_components=8, random_state=7,
  svd_solver='auto', tol=0.0, whiten=False)), ('km', KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=10, n_init=10, n_jobs=None, precompute_distances='auto',
    random_state=7, ...=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False))])

In [153]:
final_estimator

MLPClassifier(activation='relu', alpha=0.001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=True, epsilon=1e-08,
       hidden_layer_sizes=(100, 25, 100), learning_rate='constant',
       learning_rate_init=0.001, max_iter=2000, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=7, shuffle=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False)

In [154]:
best_estimator.score(training_x, training_y)

0.847255369928401

In [155]:
best_estimator.score(testing_x, testing_y)

0.8222222222222222

In [162]:
pipe = Pipeline([('pca', pca), ('gmm', gmm), ('NN', mlp)])
gs, best_estimator, final_estimator = gs_with_best_estimator(pipe, grid)

Fitting 5 folds for each of 20 candidates, totalling 100 fits
[CV] NN__alpha=0.1, NN__hidden_layer_sizes=(50, 50) ..................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  NN__alpha=0.1, NN__hidden_layer_sizes=(50, 50), score=0.6163076923076922, total=   0.3s
[CV] NN__alpha=0.1, NN__hidden_layer_sizes=(50, 50) ..................
[CV]  NN__alpha=0.1, NN__hidden_layer_sizes=(50, 50), score=0.6755384615384616, total=   0.2s
[CV] NN__alpha=0.1, NN__hidden_layer_sizes=(50, 50) ..................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.5s remaining:    0.0s


[CV]  NN__alpha=0.1, NN__hidden_layer_sizes=(50, 50), score=0.6443076923076924, total=   0.2s
[CV] NN__alpha=0.1, NN__hidden_layer_sizes=(50, 50) ..................


...

[CV]  NN__alpha=0.1, NN__hidden_layer_sizes=(50,), score=0.5960000000000001, total=   0.1s
[CV] NN__alpha=0.1, NN__hidden_layer_sizes=(50,) .....................
[CV]  NN__alpha=0.1, NN__hidden_layer_sizes=(50,), score=0.6338333333333332, total=   0.1s
[CV] NN__alpha=0.1, NN__hidden_layer_sizes=(25,) .....................


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:    1.9s remaining:    0.0s


[CV]  NN__alpha=0.1, NN__hidden_layer_sizes=(25,), score=0.09999999999999998, total=   0.1s
[CV] NN__alpha=0.1, NN__hidden_layer_sizes=(25,) .....................
[CV]  NN__alpha=0.1, NN__hidden_layer_sizes=(25,), score=0.6620000000000001, total=   0.2s
[CV] NN__alpha=0.1, NN__hidden_layer_sizes=(25,) .....................
[CV]  NN__alpha=0.1, NN__hidden_layer_sizes=(25,), score=0.1, total=   0.1s
[CV] NN__alpha=0.1, NN__hidden_layer_sizes=(25,) .....................
[CV]  NN__alpha=0.1, NN__hidden_layer_sizes=(25,), score=0.5616666666666666, total=   0.2s
[CV] NN__alpha=0.1, NN__hidden_layer_sizes=(25,) .....................
[CV]  NN__alpha=0.1, NN__hidden_layer_sizes=(25,), score=0.1, total=   0.1s
[CV] NN__alpha=0.1, NN__hidden_layer_sizes=(25, 25) ..................
[CV]  NN__alpha=0.1, NN__hidden_layer_sizes=(25, 25), score=0.607076923076923, total=   0.2s
[CV] NN__alpha=0.1, NN__hidden_layer_sizes=(25, 25) ..................
[CV]  NN__alpha=0.1, NN__hidden_layer_sizes=(25, 25), s

[CV]  NN__alpha=0.001, NN__hidden_layer_sizes=(25,), score=0.1, total=   0.1s
[CV] NN__alpha=0.001, NN__hidden_layer_sizes=(25,) ...................
[CV]  NN__alpha=0.001, NN__hidden_layer_sizes=(25,), score=0.5574999999999999, total=   0.2s
[CV] NN__alpha=0.001, NN__hidden_layer_sizes=(25,) ...................
[CV]  NN__alpha=0.001, NN__hidden_layer_sizes=(25,), score=0.1, total=   0.1s
[CV] NN__alpha=0.001, NN__hidden_layer_sizes=(25, 25) ................
[CV]  NN__alpha=0.001, NN__hidden_layer_sizes=(25, 25), score=0.607076923076923, total=   0.2s
[CV] NN__alpha=0.001, NN__hidden_layer_sizes=(25, 25) ................
[CV]  NN__alpha=0.001, NN__hidden_layer_sizes=(25, 25), score=0.6549230769230772, total=   0.2s
[CV] NN__alpha=0.001, NN__hidden_layer_sizes=(25, 25) ................
[CV]  NN__alpha=0.001, NN__hidden_layer_sizes=(25, 25), score=0.6, total=   0.2s
[CV] NN__alpha=0.001, NN__hidden_layer_sizes=(25, 25) ................
[CV]  NN__alpha=0.001, NN__hidden_layer_sizes=(25, 25

[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:   21.9s finished
/Users/frankz/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [163]:
best_estimator

Pipeline(memory=None,
     steps=[('pca', PCA(copy=True, iterated_power='auto', n_components=8, random_state=7,
  svd_solver='auto', tol=0.0, whiten=False)), ('gmm', CustomGMM(covariance_type='full', init_params='kmeans', max_iter=100,
     means_init=None, n_components=10, n_init=1, precisions_init=None,
     random_state=7,...=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False))])

In [164]:
final_estimator

MLPClassifier(activation='relu', alpha=0.001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=True, epsilon=1e-08,
       hidden_layer_sizes=(50, 50), learning_rate='constant',
       learning_rate_init=0.001, max_iter=2000, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=7, shuffle=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False)

In [165]:
best_estimator.score(training_x, training_y)

0.669848846459825

In [166]:
best_estimator.score(testing_x, testing_y)

0.6481481481481481

### ICA

In [169]:
from sklearn.decomposition import FastICA

In [170]:
ica = FastICA(n_components=13, random_state=seed)

In [171]:
pipe = Pipeline([('ica', ica), ('km', km), ('NN', mlp)])
gs, best_estimator, final_estimator = gs_with_best_estimator(pipe, grid)

Fitting 5 folds for each of 20 candidates, totalling 100 fits
[CV] NN__alpha=0.1, NN__hidden_layer_sizes=(50, 50) ..................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/Users/frankz/anaconda3/lib/python3.7/site-packages/sklearn/decomposition/fastica_.py:121: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  ConvergenceWarning)


[CV]  NN__alpha=0.1, NN__hidden_layer_sizes=(50, 50), score=0.45661538461538453, total=   0.5s
[CV] NN__alpha=0.1, NN__hidden_layer_sizes=(50, 50) ..................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.5s remaining:    0.0s
/Users/frankz/anaconda3/lib/python3.7/site-packages/sklearn/decomposition/fastica_.py:121: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  ConvergenceWarning)


[CV]  NN__alpha=0.1, NN__hidden_layer_sizes=(50, 50), score=0.18061538461538462, total=   0.3s
[CV] NN__alpha=0.1, NN__hidden_layer_sizes=(50, 50) ..................


...

/Users/frankz/anaconda3/lib/python3.7/site-packages/sklearn/decomposition/fastica_.py:121: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  ConvergenceWarning)


[CV]  NN__alpha=0.0001, NN__hidden_layer_sizes=(100, 25, 100), score=0.5743461538461538, total=   1.1s
[CV] NN__alpha=0.0001, NN__hidden_layer_sizes=(100, 25, 100) .........


/Users/frankz/anaconda3/lib/python3.7/site-packages/sklearn/decomposition/fastica_.py:121: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  ConvergenceWarning)


[CV]  NN__alpha=0.0001, NN__hidden_layer_sizes=(100, 25, 100), score=0.212, total=   0.4s
[CV] NN__alpha=0.0001, NN__hidden_layer_sizes=(100, 25, 100) .........
[CV]  NN__alpha=0.0001, NN__hidden_layer_sizes=(100, 25, 100), score=0.6153333333333334, total=   1.0s


[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:   42.9s finished
/Users/frankz/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
/Users/frankz/anaconda3/lib/python3.7/site-packages/sklearn/decomposition/fastica_.py:121: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  ConvergenceWarning)
/Users/frankz/anaconda3/lib/python3.7/site-packages/sklearn/decomposition/fastica_.py:121: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  ConvergenceWarning)


In [172]:
best_estimator

Pipeline(memory=None,
     steps=[('ica', FastICA(algorithm='parallel', fun='logcosh', fun_args=None, max_iter=200,
    n_components=13, random_state=7, tol=0.0001, w_init=None, whiten=True)), ('km', KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=10, n_init=10, n_jobs=None, precompute_dis...=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False))])

In [173]:
final_estimator

MLPClassifier(activation='relu', alpha=0.01, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=True, epsilon=1e-08,
       hidden_layer_sizes=(100, 25, 100), learning_rate='constant',
       learning_rate_init=0.001, max_iter=2000, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=7, shuffle=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False)

In [174]:
best_estimator.score(training_x, training_y)

0.10103420843277645

In [175]:
best_estimator.score(testing_x, testing_y)

0.10185185185185185

In [176]:
pipe = Pipeline([('ica', ica), ('gmm', gmm), ('NN', mlp)])
gs, best_estimator, final_estimator = gs_with_best_estimator(pipe, grid)

Fitting 5 folds for each of 20 candidates, totalling 100 fits
[CV] NN__alpha=0.1, NN__hidden_layer_sizes=(50, 50) ..................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/Users/frankz/anaconda3/lib/python3.7/site-packages/sklearn/decomposition/fastica_.py:121: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  ConvergenceWarning)


[CV]  NN__alpha=0.1, NN__hidden_layer_sizes=(50, 50), score=0.621076923076923, total=   0.4s
[CV] NN__alpha=0.1, NN__hidden_layer_sizes=(50, 50) ..................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.4s remaining:    0.0s
/Users/frankz/anaconda3/lib/python3.7/site-packages/sklearn/decomposition/fastica_.py:121: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  ConvergenceWarning)


[CV]  NN__alpha=0.1, NN__hidden_layer_sizes=(50, 50), score=0.6610769230769232, total=   0.4s
[CV] NN__alpha=0.1, NN__hidden_layer_sizes=(50, 50) ..................


...

/Users/frankz/anaconda3/lib/python3.7/site-packages/sklearn/decomposition/fastica_.py:121: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  ConvergenceWarning)


[CV]  NN__alpha=0.0001, NN__hidden_layer_sizes=(100, 25, 100), score=0.6490769230769231, total=   0.5s
[CV] NN__alpha=0.0001, NN__hidden_layer_sizes=(100, 25, 100) .........


/Users/frankz/anaconda3/lib/python3.7/site-packages/sklearn/decomposition/fastica_.py:121: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  ConvergenceWarning)


[CV]  NN__alpha=0.0001, NN__hidden_layer_sizes=(100, 25, 100), score=0.6183333333333334, total=   0.7s
[CV] NN__alpha=0.0001, NN__hidden_layer_sizes=(100, 25, 100) .........
[CV]  NN__alpha=0.0001, NN__hidden_layer_sizes=(100, 25, 100), score=0.6570000000000001, total=   0.3s


[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:   35.7s finished
/Users/frankz/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
/Users/frankz/anaconda3/lib/python3.7/site-packages/sklearn/decomposition/fastica_.py:121: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  ConvergenceWarning)
/Users/frankz/anaconda3/lib/python3.7/site-packages/sklearn/decomposition/fastica_.py:121: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  ConvergenceWarning)


In [177]:
best_estimator

Pipeline(memory=None,
     steps=[('ica', FastICA(algorithm='parallel', fun='logcosh', fun_args=None, max_iter=200,
    n_components=13, random_state=7, tol=0.0001, w_init=None, whiten=True)), ('gmm', CustomGMM(covariance_type='full', init_params='kmeans', max_iter=100,
     means_init=None, n_components=10, n_init=1, precisi...=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False))])

In [178]:
final_estimator

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=True, epsilon=1e-08,
       hidden_layer_sizes=(100, 25, 100), learning_rate='constant',
       learning_rate_init=0.001, max_iter=2000, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=7, shuffle=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False)

In [179]:
best_estimator.score(training_x, training_y)

0.6292760540970564

In [180]:
best_estimator.score(testing_x, testing_y)

0.6481481481481481

### RP

In [123]:
from sklearn.random_projection import SparseRandomProjection, GaussianRandomProjection

In [124]:
rp = SparseRandomProjection(n_components=9, random_state=seed)

In [181]:
pipe = Pipeline([('rp', rp), ('km', km), ('NN', mlp)])
gs, best_estimator, final_estimator = gs_with_best_estimator(pipe, grid)

Fitting 5 folds for each of 20 candidates, totalling 100 fits
[CV] NN__alpha=0.1, NN__hidden_layer_sizes=(50, 50) ..................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  NN__alpha=0.1, NN__hidden_layer_sizes=(50, 50), score=0.5863076923076923, total=   0.6s
[CV] NN__alpha=0.1, NN__hidden_layer_sizes=(50, 50) ..................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.6s remaining:    0.0s


[CV]  NN__alpha=0.1, NN__hidden_layer_sizes=(50, 50), score=0.6218461538461539, total=   0.5s
[CV] NN__alpha=0.1, NN__hidden_layer_sizes=(50, 50) ..................


...

[CV]  NN__alpha=0.1, NN__hidden_layer_sizes=(50,), score=0.48566666666666664, total=   0.3s
[CV] NN__alpha=0.1, NN__hidden_layer_sizes=(50,) .....................


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:    3.8s remaining:    0.0s


[CV]  NN__alpha=0.1, NN__hidden_layer_sizes=(50,), score=0.5315, total=   0.4s
[CV] NN__alpha=0.1, NN__hidden_layer_sizes=(25,) .....................
[CV]  NN__alpha=0.1, NN__hidden_layer_sizes=(25,), score=0.15584615384615383, total=   0.2s
[CV] NN__alpha=0.1, NN__hidden_layer_sizes=(25,) .....................
[CV]  NN__alpha=0.1, NN__hidden_layer_sizes=(25,), score=0.32030769230769235, total=   0.3s
[CV] NN__alpha=0.1, NN__hidden_layer_sizes=(25,) .....................
[CV]  NN__alpha=0.1, NN__hidden_layer_sizes=(25,), score=0.12016666666666667, total=   0.2s
[CV] NN__alpha=0.1, NN__hidden_layer_sizes=(25,) .....................
[CV]  NN__alpha=0.1, NN__hidden_layer_sizes=(25,), score=0.14316666666666664, total=   0.1s
[CV] NN__alpha=0.1, NN__hidden_layer_sizes=(25,) .....................
[CV]  NN__alpha=0.1, NN__hidden_layer_sizes=(25,), score=0.15333333333333335, total=   0.2s
[CV] NN__alpha=0.1, NN__hidden_layer_sizes=(25, 25) ..................
[CV]  NN__alpha=0.1, NN__hidden_lay

[CV]  NN__alpha=0.001, NN__hidden_layer_sizes=(50,), score=0.5275, total=   0.3s
[CV] NN__alpha=0.001, NN__hidden_layer_sizes=(25,) ...................
[CV]  NN__alpha=0.001, NN__hidden_layer_sizes=(25,), score=0.09430769230769229, total=   0.1s
[CV] NN__alpha=0.001, NN__hidden_layer_sizes=(25,) ...................
[CV]  NN__alpha=0.001, NN__hidden_layer_sizes=(25,), score=0.32030769230769235, total=   0.2s
[CV] NN__alpha=0.001, NN__hidden_layer_sizes=(25,) ...................
[CV]  NN__alpha=0.001, NN__hidden_layer_sizes=(25,), score=0.12016666666666667, total=   0.2s
[CV] NN__alpha=0.001, NN__hidden_layer_sizes=(25,) ...................
[CV]  NN__alpha=0.001, NN__hidden_layer_sizes=(25,), score=0.14316666666666664, total=   0.1s
[CV] NN__alpha=0.001, NN__hidden_layer_sizes=(25,) ...................
[CV]  NN__alpha=0.001, NN__hidden_layer_sizes=(25,), score=0.15333333333333335, total=   0.2s
[CV] NN__alpha=0.001, NN__hidden_layer_sizes=(25, 25) ................
[CV]  NN__alpha=0.001, 

[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:   37.7s finished
/Users/frankz/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [182]:
best_estimator

Pipeline(memory=None,
     steps=[('rp', SparseRandomProjection(dense_output=False, density='auto', eps=0.1,
            n_components=9, random_state=7)), ('km', KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=10, n_init=10, n_jobs=None, precompute_distances='auto',
    random_state=7, tol...=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False))])

In [183]:
final_estimator

MLPClassifier(activation='relu', alpha=0.001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=True, epsilon=1e-08,
       hidden_layer_sizes=(100, 25, 100), learning_rate='constant',
       learning_rate_init=0.001, max_iter=2000, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=7, shuffle=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False)

In [184]:
best_estimator.score(training_x, training_y)

0.6523468575974543

In [185]:
best_estimator.score(testing_x, testing_y)

0.6166666666666667

In [186]:
pipe = Pipeline([('rp', rp), ('gmm', gmm), ('NN', mlp)])
gs, best_estimator, final_estimator = gs_with_best_estimator(pipe, grid)

Fitting 5 folds for each of 20 candidates, totalling 100 fits
[CV] NN__alpha=0.1, NN__hidden_layer_sizes=(50, 50) ..................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  NN__alpha=0.1, NN__hidden_layer_sizes=(50, 50), score=0.5455384615384614, total=   0.4s
[CV] NN__alpha=0.1, NN__hidden_layer_sizes=(50, 50) ..................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.4s remaining:    0.0s


[CV]  NN__alpha=0.1, NN__hidden_layer_sizes=(50, 50), score=0.5226153846153848, total=   0.5s
[CV] NN__alpha=0.1, NN__hidden_layer_sizes=(50, 50) ..................


...

[CV]  NN__alpha=0.1, NN__hidden_layer_sizes=(50,), score=0.472, total=   0.3s
[CV] NN__alpha=0.1, NN__hidden_layer_sizes=(50,) .....................
[CV]  NN__alpha=0.1, NN__hidden_layer_sizes=(50,), score=0.45366666666666666, total=   0.2s
[CV] NN__alpha=0.1, NN__hidden_layer_sizes=(25,) .....................


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:    3.2s remaining:    0.0s


[CV]  NN__alpha=0.1, NN__hidden_layer_sizes=(25,), score=0.5186153846153846, total=   0.3s
[CV] NN__alpha=0.1, NN__hidden_layer_sizes=(25,) .....................
[CV]  NN__alpha=0.1, NN__hidden_layer_sizes=(25,), score=0.49307692307692325, total=   0.3s
[CV] NN__alpha=0.1, NN__hidden_layer_sizes=(25,) .....................
[CV]  NN__alpha=0.1, NN__hidden_layer_sizes=(25,), score=0.45621794871794874, total=   0.2s
[CV] NN__alpha=0.1, NN__hidden_layer_sizes=(25,) .....................
[CV]  NN__alpha=0.1, NN__hidden_layer_sizes=(25,), score=0.5208333333333333, total=   0.2s
[CV] NN__alpha=0.1, NN__hidden_layer_sizes=(25,) .....................
[CV]  NN__alpha=0.1, NN__hidden_layer_sizes=(25,), score=0.13550000000000004, total=   0.1s
[CV] NN__alpha=0.1, NN__hidden_layer_sizes=(25, 25) ..................
[CV]  NN__alpha=0.1, NN__hidden_layer_sizes=(25, 25), score=0.5109230769230769, total=   0.3s
[CV] NN__alpha=0.1, NN__hidden_layer_sizes=(25, 25) ..................
[CV]  NN__alpha=0.1, N

[CV]  NN__alpha=0.001, NN__hidden_layer_sizes=(25,), score=0.5226153846153846, total=   0.2s
[CV] NN__alpha=0.001, NN__hidden_layer_sizes=(25,) ...................
[CV]  NN__alpha=0.001, NN__hidden_layer_sizes=(25,), score=0.5004615384615386, total=   0.2s
[CV] NN__alpha=0.001, NN__hidden_layer_sizes=(25,) ...................
[CV]  NN__alpha=0.001, NN__hidden_layer_sizes=(25,), score=0.46006410256410263, total=   0.2s
[CV] NN__alpha=0.001, NN__hidden_layer_sizes=(25,) ...................
[CV]  NN__alpha=0.001, NN__hidden_layer_sizes=(25,), score=0.501, total=   0.2s
[CV] NN__alpha=0.001, NN__hidden_layer_sizes=(25,) ...................
[CV]  NN__alpha=0.001, NN__hidden_layer_sizes=(25,), score=0.13550000000000004, total=   0.1s
[CV] NN__alpha=0.001, NN__hidden_layer_sizes=(25, 25) ................
[CV]  NN__alpha=0.001, NN__hidden_layer_sizes=(25, 25), score=0.5149230769230769, total=   0.3s
[CV] NN__alpha=0.001, NN__hidden_layer_sizes=(25, 25) ................
[CV]  NN__alpha=0.001, N

[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:   30.3s finished
/Users/frankz/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [187]:
best_estimator

Pipeline(memory=None,
     steps=[('rp', SparseRandomProjection(dense_output=False, density='auto', eps=0.1,
            n_components=9, random_state=7)), ('gmm', CustomGMM(covariance_type='full', init_params='kmeans', max_iter=100,
     means_init=None, n_components=10, n_init=1, precisions_init=None,
     random_state=7, re...=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False))])

In [188]:
final_estimator

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=True, epsilon=1e-08,
       hidden_layer_sizes=(50, 50), learning_rate='constant',
       learning_rate_init=0.001, max_iter=2000, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=7, shuffle=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False)

In [189]:
best_estimator.score(training_x, training_y)

0.5608591885441527

In [190]:
best_estimator.score(testing_x, testing_y)

0.5574074074074075

### RF

In [191]:
from sklearn.ensemble import RandomForestClassifier

In [192]:
from sklearn.base import TransformerMixin,BaseEstimator

In [193]:
class ImportanceSelect(BaseEstimator, TransformerMixin):
    def __init__(self, model, n=1):
        self.model = model
        self.n = n

    def fit(self, *args, **kwargs):
        self.model.fit(*args, **kwargs)
        return self

    def transform(self, X):
        return X[:, self.model.feature_importances_.argsort()[::-1][:self.n]]

In [194]:
rfc = RandomForestClassifier(n_estimators=100, class_weight='balanced', random_state=seed)
filtr = ImportanceSelect(rfc, 11)

In [195]:
pipe = Pipeline([('filtr', filtr), ('km', km), ('NN', mlp)])
gs, best_estimator, final_estimator = gs_with_best_estimator(pipe, grid)

Fitting 5 folds for each of 20 candidates, totalling 100 fits
[CV] NN__alpha=0.1, NN__hidden_layer_sizes=(50, 50) ..................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  NN__alpha=0.1, NN__hidden_layer_sizes=(50, 50), score=0.793076923076923, total=   0.9s
[CV] NN__alpha=0.1, NN__hidden_layer_sizes=(50, 50) ..................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.9s remaining:    0.0s


[CV]  NN__alpha=0.1, NN__hidden_layer_sizes=(50, 50), score=0.7613846153846155, total=   0.6s
[CV] NN__alpha=0.1, NN__hidden_layer_sizes=(50, 50) ..................


...

[CV]  NN__alpha=0.1, NN__hidden_layer_sizes=(50,), score=0.7581666666666667, total=   0.5s
[CV] NN__alpha=0.1, NN__hidden_layer_sizes=(50,) .....................


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:    5.3s remaining:    0.0s


[CV]  NN__alpha=0.1, NN__hidden_layer_sizes=(50,), score=0.6633333333333333, total=   0.4s
[CV] NN__alpha=0.1, NN__hidden_layer_sizes=(25,) .....................
[CV]  NN__alpha=0.1, NN__hidden_layer_sizes=(25,), score=0.09999999999999998, total=   0.4s
[CV] NN__alpha=0.1, NN__hidden_layer_sizes=(25,) .....................
[CV]  NN__alpha=0.1, NN__hidden_layer_sizes=(25,), score=0.19200000000000003, total=   0.4s
[CV] NN__alpha=0.1, NN__hidden_layer_sizes=(25,) .....................
[CV]  NN__alpha=0.1, NN__hidden_layer_sizes=(25,), score=0.2915, total=   0.4s
[CV] NN__alpha=0.1, NN__hidden_layer_sizes=(25,) .....................
[CV]  NN__alpha=0.1, NN__hidden_layer_sizes=(25,), score=0.6735, total=   0.6s
[CV] NN__alpha=0.1, NN__hidden_layer_sizes=(25,) .....................
[CV]  NN__alpha=0.1, NN__hidden_layer_sizes=(25,), score=0.19166666666666665, total=   0.4s
[CV] NN__alpha=0.1, NN__hidden_layer_sizes=(25, 25) ..................
[CV]  NN__alpha=0.1, NN__hidden_layer_sizes=(25, 

[CV]  NN__alpha=0.001, NN__hidden_layer_sizes=(50,), score=0.6673333333333333, total=   0.5s
[CV] NN__alpha=0.001, NN__hidden_layer_sizes=(25,) ...................
[CV]  NN__alpha=0.001, NN__hidden_layer_sizes=(25,), score=0.09999999999999998, total=   0.3s
[CV] NN__alpha=0.001, NN__hidden_layer_sizes=(25,) ...................
[CV]  NN__alpha=0.001, NN__hidden_layer_sizes=(25,), score=0.19200000000000003, total=   0.4s
[CV] NN__alpha=0.001, NN__hidden_layer_sizes=(25,) ...................
[CV]  NN__alpha=0.001, NN__hidden_layer_sizes=(25,), score=0.2915, total=   0.4s
[CV] NN__alpha=0.001, NN__hidden_layer_sizes=(25,) ...................
[CV]  NN__alpha=0.001, NN__hidden_layer_sizes=(25,), score=0.653, total=   0.6s
[CV] NN__alpha=0.001, NN__hidden_layer_sizes=(25,) ...................
[CV]  NN__alpha=0.001, NN__hidden_layer_sizes=(25,), score=0.19166666666666665, total=   0.4s
[CV] NN__alpha=0.001, NN__hidden_layer_sizes=(25, 25) ................
[CV]  NN__alpha=0.001, NN__hidden_laye

[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:   57.3s finished
/Users/frankz/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [196]:
best_estimator

Pipeline(memory=None,
     steps=[('filtr', ImportanceSelect(model=RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='gini', max_depth=None, max_features='auto',
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            ...=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False))])

In [197]:
final_estimator

MLPClassifier(activation='relu', alpha=0.1, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=True, epsilon=1e-08,
       hidden_layer_sizes=(100, 25, 100), learning_rate='constant',
       learning_rate_init=0.001, max_iter=2000, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=7, shuffle=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False)

In [198]:
best_estimator.score(training_x, training_y)

0.8687350835322196

In [199]:
best_estimator.score(testing_x, testing_y)

0.8611111111111112

In [200]:
pipe = Pipeline([('filtr', filtr), ('gmm', gmm), ('NN', mlp)])
gs, best_estimator, final_estimator = gs_with_best_estimator(pipe, grid)

Fitting 5 folds for each of 20 candidates, totalling 100 fits
[CV] NN__alpha=0.1, NN__hidden_layer_sizes=(50, 50) ..................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  NN__alpha=0.1, NN__hidden_layer_sizes=(50, 50), score=0.5758461538461538, total=   0.5s
[CV] NN__alpha=0.1, NN__hidden_layer_sizes=(50, 50) ..................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.5s remaining:    0.0s


[CV]  NN__alpha=0.1, NN__hidden_layer_sizes=(50, 50), score=0.6001538461538463, total=   0.5s
[CV] NN__alpha=0.1, NN__hidden_layer_sizes=(50, 50) ..................


...

[CV]  NN__alpha=0.1, NN__hidden_layer_sizes=(50,), score=0.5128333333333334, total=   0.3s
[CV] NN__alpha=0.1, NN__hidden_layer_sizes=(50,) .....................


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:    4.1s remaining:    0.0s


[CV]  NN__alpha=0.1, NN__hidden_layer_sizes=(50,), score=0.516, total=   0.4s
[CV] NN__alpha=0.1, NN__hidden_layer_sizes=(25,) .....................
[CV]  NN__alpha=0.1, NN__hidden_layer_sizes=(25,), score=0.4193846153846153, total=   0.4s
[CV] NN__alpha=0.1, NN__hidden_layer_sizes=(25,) .....................
[CV]  NN__alpha=0.1, NN__hidden_layer_sizes=(25,), score=0.6050769230769232, total=   0.4s
[CV] NN__alpha=0.1, NN__hidden_layer_sizes=(25,) .....................
[CV]  NN__alpha=0.1, NN__hidden_layer_sizes=(25,), score=0.4878333333333334, total=   0.4s
[CV] NN__alpha=0.1, NN__hidden_layer_sizes=(25,) .....................
[CV]  NN__alpha=0.1, NN__hidden_layer_sizes=(25,), score=0.09999999999999998, total=   0.3s
[CV] NN__alpha=0.1, NN__hidden_layer_sizes=(25,) .....................
[CV]  NN__alpha=0.1, NN__hidden_layer_sizes=(25,), score=0.15950000000000003, total=   0.5s
[CV] NN__alpha=0.1, NN__hidden_layer_sizes=(25, 25) ..................
[CV]  NN__alpha=0.1, NN__hidden_layer_s

[CV]  NN__alpha=0.001, NN__hidden_layer_sizes=(50,), score=0.5805, total=   0.5s
[CV] NN__alpha=0.001, NN__hidden_layer_sizes=(25,) ...................
[CV]  NN__alpha=0.001, NN__hidden_layer_sizes=(25,), score=0.4193846153846153, total=   0.4s
[CV] NN__alpha=0.001, NN__hidden_layer_sizes=(25,) ...................
[CV]  NN__alpha=0.001, NN__hidden_layer_sizes=(25,), score=0.6050769230769232, total=   0.4s
[CV] NN__alpha=0.001, NN__hidden_layer_sizes=(25,) ...................
[CV]  NN__alpha=0.001, NN__hidden_layer_sizes=(25,), score=0.49183333333333334, total=   0.4s
[CV] NN__alpha=0.001, NN__hidden_layer_sizes=(25,) ...................
[CV]  NN__alpha=0.001, NN__hidden_layer_sizes=(25,), score=0.09999999999999998, total=   0.3s
[CV] NN__alpha=0.001, NN__hidden_layer_sizes=(25,) ...................
[CV]  NN__alpha=0.001, NN__hidden_layer_sizes=(25,), score=0.15950000000000003, total=   0.3s
[CV] NN__alpha=0.001, NN__hidden_layer_sizes=(25, 25) ................
[CV]  NN__alpha=0.001, NN

[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:   44.8s finished
/Users/frankz/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [201]:
best_estimator

Pipeline(memory=None,
     steps=[('filtr', ImportanceSelect(model=RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='gini', max_depth=None, max_features='auto',
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            ...=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False))])

In [202]:
final_estimator

MLPClassifier(activation='relu', alpha=0.1, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=True, epsilon=1e-08,
       hidden_layer_sizes=(100, 25, 100), learning_rate='constant',
       learning_rate_init=0.001, max_iter=2000, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=7, shuffle=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False)

In [203]:
best_estimator.score(training_x, training_y)

0.5600636435958631

In [204]:
best_estimator.score(testing_x, testing_y)

0.5037037037037037